<a href="https://colab.research.google.com/github/ValdazoAmerico/word2vec/blob/main/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
# Load my word embeddings
import gensim
wordembeddings = gensim.models.KeyedVectors.load_word2vec_format('drive/My Drive/GoogleNews-vectors-negative300.bin', binary=True)

In [4]:
x = ["I am very happy today", "No, I do not like the movie"]
y = [1,0]

In [8]:
#Create a tokenizer, configured to only take into account the 20 most common words
tokenizer = Tokenizer(num_words=20)

In [9]:
#Build the word index
tokenizer.fit_on_texts(x)

In [10]:
sequences = tokenizer.texts_to_sequences(x)

In [11]:
sequences

[[1, 2, 3, 4, 5], [6, 1, 7, 8, 9, 10, 11]]

In [12]:
word_index = tokenizer.word_index
print(tokenizer.word_index)
print('Found %s unique tokens.' % len(word_index))

{'i': 1, 'am': 2, 'very': 3, 'happy': 4, 'today': 5, 'no': 6, 'do': 7, 'not': 8, 'like': 9, 'the': 10, 'movie': 11}
Found 11 unique tokens.


In [13]:
max_length = 0
for review_number in range(len(sequences)):
  numberofwords=len(sequences[review_number])
  if (numberofwords) > (max_length):
    max_length = numberofwords
print(max_length)
 

7


In [14]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
data = pad_sequences(sequences,maxlen=max_length)
y =  np.asarray(y)
print(data)
print('Shape of data tensor:',data.shape)
print('Shape of label tensor:',y.shape)

[[ 0  0  1  2  3  4  5]
 [ 6  1  7  8  9 10 11]]
Shape of data tensor: (2, 7)
Shape of label tensor: (2,)


In [16]:
unique_words = len(word_index)
total_words = unique_words + 1
skipped_words = 0
embedding_dim = 300
embedding_matrix = np.zeros((total_words, embedding_dim))
for word, index in tokenizer.word_index.items():
  try:
    embedding_vector = wordembeddings[word]
  except:
    skipped_words = skipped_words+1
    pass
  if embedding_vector is not None:
    embedding_matrix[index] = embedding_vector
print("Embeddings Matrix Shape:", embedding_matrix.shape)


Embeddings Matrix Shape: (12, 300)


In [17]:
embedding_matrix[3] - wordembeddings.wv['very']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [19]:
from tensorflow.keras.layers import *

embedding_layer = Embedding(total_words, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False)

In [20]:
from tensorflow.keras.models import *

model = Sequential()
model.add(embedding_layer)
model.add(SimpleRNN(128,activation='relu',return_sequences=True))
model.add(SimpleRNN(256,activation='relu'))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 7, 300)            3600      
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 7, 128)            54912     
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 256)               98560     
_________________________________________________________________
dense (Dense)                (None, 1)                 257       
Total params: 157,329
Trainable params: 153,729
Non-trainable params: 3,600
_________________________________________________________________
None


In [23]:
from keras.optimizers import *

opt = SGD(lr=0.01,decay=1e-6)
model.compile(loss = "binary_crossentropy", optimizer=opt,metrics=['accuracy'])
model.fit(data, y, epochs=10, verbose=1)

Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 0.6693 - accuracy: 0.5000
Epoch 2/10
1/1 [==============================] - 0s 14ms/step - loss: 0.6515 - accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - 0s 16ms/step - loss: 0.6347 - accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 15ms/step - loss: 0.6193 - accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 0s 10ms/step - loss: 0.6045 - accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 13ms/step - loss: 0.5898 - accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 13ms/step - loss: 0.5760 - accuracy: 1.0000
Epoch 8/10
1/1 [==============================] - 0s 9ms/step - loss: 0.5628 - accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - 0s 14ms/step - loss: 0.5505 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 0s 11ms/step - loss: 0.5389 - accuracy: 1.0000
